Mishael Derla, FAU Erlangen-Nürnberg, winter term 2024/2025

In [2]:
from os import listdir

import numpy
from scipy.special import gamma as euler_gamma_function

import matplotlib as mpl
import matplotlib.pyplot as plt

from xyz_file_io import XYZ, active_particle_metadata
from physics import pbc_iterator

Our $n$-body alignment distribution function reads
$$\boldsymbol{a}^{(n)}(\vec{r}_1,\dots,\vec{r}_n)=\left\langle\;\sum_{i_1}\sum_{i_1\ne i_2}\cdots\sum_{i_n\ne i_1,i_2,\dots,i_{n-1}}\frac{\vec{F}^\text{(self)}_{i_1}}{F^\text{(self)}}\frac{\delta(\vec{r}_1-\vec{x}_{i_1})}{\rho(\vec{r}_1)}\otimes\cdots\otimes\frac{\vec{F}^\text{(self)}_{i_n}}{F^\text{(self)}}\frac{\delta(\vec{r}_1-\vec{x}_{i_n})}{\rho(\vec{r}_n)}\;\right\rangle$$
When the system is homogeneous and isotropic (which our system would be for $N,V\to\infty$ as $N/V=:\rho$ remains constant), this reduces to a scalar $a{(2)}(r)$, which is approximated by the following binning
$$a^{(2)}(r)\approx\frac{1}{Nd}\cdot\frac{1}{S_dr^{d-1}\Delta r\,\rho}\cdot\underbrace{\left\langle\sum_i\sum_{j\ne i}\frac{\vec{F}_i^\text{(self)}}{F^\text{(self)}}\cdot\frac{\vec{F}_j^\text{(self)}}{F^\text{(self)}}\,\boldsymbol{1}_{|\vec{x}_i-\vec{x}_j|\in[r,r+\Delta r]}\right\rangle}_{\substack{\text{average over the sum of all pair alignments}\\ \vec{F}_i^\text{(self)}\vec{F}_j^\text{(self)}/(F^\text{(self)})^2\text{ such that $|\vec{x}_i-\vec{x}_j|\in[r,r+\Delta r]$}}}$$
which we have implemented below

In [3]:
# TODO: pair correlation tensor? but how justify mere r dependence?
def pair_alignment_radial_binning(
		positions: numpy.ndarray,
		self_propulsions: numpy.ndarray,
		box_sidelength:float,
		bins:int=10
		) -> tuple[numpy.ndarray, list[float], list[float], list[float]]:
	'''
	Computes Alignment(r) and its error; also computes a
	distance histogtam Distances(r) for reference

	Args:
		positions:
			numpy.ndarray of shape (N,d) containing the
			positions of N particles in d spatial dimensions
		self_propulsions:
			numpy.ndarray of shape (N,d) containint the
			components of the self propulsion force of
			N particles in d spatial dimensions
		box_sidelength:
			domain sidelength, the distance traveled before
			wrapping around in periodic boundary conditions
		bins:
			the number of radial bins (indexing the 'bags',
			i.e. arbitrarily sized lists of alignment values)
	
	Returns:
		tuple containing, in the following order:
			0. the radial bin lower edges
			1. a distance histogram (for reference)
			2. a list of alignment values
			3. a list of alignment value errors
	'''
	particle_number, spatial_dimensions = positions.shape

	# normalized self-propulsions
	normalized_self_propulsions = numpy.array([
		propulsion / numpy.linalg.norm(propulsion)
		for propulsion in self_propulsions
	])

	# binning from 0 to box_sidelength
	highest_meaningful_distance = box_sidelength / 2
	bin_lower_edges = numpy.linspace(1e-5, highest_meaningful_distance, bins)
	alignment_bags = [[] for _ in range(bins)]
	distance_histogram = [0] * bins
		
	# looping over all unique ...
	for i in range(particle_number):
		# ... pairs of particles ...
		for j in range(particle_number):

			# no measuring of self alignment
			if i == j:
				continue

			# for all periodic images
			for pbc_image in pbc_iterator(spatial_dimensions):

				# distance of i and j squared, including periodic images
				separation_ij = positions[j] - positions[i] + box_sidelength * numpy.array(pbc_image)
				r_ij = numpy.linalg.norm(separation_ij)

				# only consider alignment up until periodic boundary ...
				# ... box ends
				if r_ij > highest_meaningful_distance:
					continue

				# bin index
				r_ij_bin_index = int(bins * r_ij / highest_meaningful_distance)

				# appending the alignment 
				a_ij = numpy.dot(normalized_self_propulsions[i], normalized_self_propulsions[j])
				alignment_bags[r_ij_bin_index].append(a_ij)

				# increasing the counter of the distance histogram
				distance_histogram[r_ij_bin_index] += 1

	return (
		bin_lower_edges,
		distance_histogram,
		[
			numpy.sum(bag) if bag else 0 # only take average if the bag has content
			for bag in alignment_bags 
		]
	)


def hypershell_volume(
		radii: numpy.ndarray,
		spatial_dimensions: float,
		shell_thickness: float
	):
	'''
	Computes the volume of a hypershell of given thickness

	Args:
		radii:
			The radii that hypershell volume should be
			computed
		spatial_dimensions:
			number of spatial dimensions
		shell_thickness:
			the shell thickness dr

	Returns:
		numpy.ndarray of hypershell volumes at the passed radii
	'''
	# d-dimensional surface
	d = spatial_dimensions
	dr = shell_thickness
	unit_shell = 2*numpy.pi**(d/2) / euler_gamma_function(d/2)
	return unit_shell * radii**(d-1) * dr


def alignment_correlation_function(
		positions: numpy.ndarray,
		self_propulsions: numpy.ndarray,
		box_sidelength:float,
		bins:int=10
	):
	'''
	Computes the alignment correlation function a^(2)(r) and its error.

	Args:
		positions:
			numpy.ndarray of shape (N,d) containing the
			positions of N particles in d spatial dimensions
		self_propulsions:
			numpy.ndarray of shape (N,d) containint the
			components of the self propulsion force of
			N particles in d spatial dimensions
		box_sidelength:
			domain sidelength, the distance traveled before
			wrapping around in periodic boundary conditions
		bins:
			the number of radial bins (indexing the 'bags',
			i.e. arbitrarily sized lists of alignment values)

	Returns:
		
	'''

	bin_lower_edges, distance_histogram, alignment_sums = pair_alignment_radial_binning(
		positions,
		self_propulsions,
		box_sidelength,
		bins=bins
	)

	# bin size
	shell_thickness = bin_lower_edges[1] - bin_lower_edges[0]

	# unpacking particle number and number of ...
	# ... spatial dimensions of the domain
	N, d = positions.shape

	hyper_shell_volumes = hypershell_volume(
		bin_lower_edges,
		d,
		shell_thickness
	)

	# computing the correct prefactor
	L = box_sidelength
	rho = N / L**d

	# number of tuples (i,j) considered
	pair_count = N * (N - 1)

	return (
		bin_lower_edges,
		distance_histogram / hyper_shell_volumes / pair_count,
		alignment_sums / (N*d*rho * hyper_shell_volumes)
	)



# Dependence of $a^{(2)}(r)$ on packing fraction $\phi$

In [49]:
# which packing fraction to render
PHI_STUDY = 0.1

In [ ]:
data_directory = f'alignment_study_datasets/phi={PHI_STUDY}'

xyz_files = {
	# simulation index : file path
	int(file_name.split('_')[2]) : f'{data_directory}/{file_name}'
	for file_name in listdir(data_directory)
	if file_name.endswith('.xyz')
}

alignment_correlations = []
radial_distributions = []

for i, path in xyz_files.items():

	if i == 0:
		continue
    
	print(f'({i}) Analyzing {path}')

	metadata = active_particle_metadata(path)

	for index, block in enumerate(XYZ.file_iterator(path)):

		# try not to get time correlation too involved. Also wait
		# ... until the simulation has become stationary.
		sampling_rate = 100
		burn_in_time = 1000
		if not (burn_in_time < index and index % sampling_rate == 0):
			continue

		# data unpacking. NOTE: assumes d=2
		positions, self_propulsions = block.xyz_data[:,(0,1)], block.xyz_data[:,(2,3)]

		N, d = positions.shape

		r, g, a = alignment_correlation_function(
			positions,
			self_propulsions,
			metadata['box_sidelength'],
			bins=100
		)
		
		alignment_correlations.append(a)
		radial_distributions.append(g)

In [ ]:
a, g = numpy.mean(alignment_correlations, axis=0), numpy.mean(radial_distributions, axis=0)
sigma = metadata['particle_diameter']

def triangular_lattice_vector_length(j, k):
	'''
	Returns the length of triangular lattice vectors
	j * a_1 + k * a_2, where a_1 = (1,0) and a_2 is
	rotated counter-clockwise by 120deg
	'''
	tri_a_1 = numpy.array([1, 0])
	tri_a_2 = numpy.array([numpy.cos(2*numpy.pi * 120/360), numpy.sin(2*numpy.pi * 120/360)])

	return numpy.linalg.norm(j * tri_a_1 + k * tri_a_2)

if PHI_STUDY >= 0.8:
	for spike_index, spike in enumerate([
		triangular_lattice_vector_length(1,0),
		triangular_lattice_vector_length(1,1),
		triangular_lattice_vector_length(2,1),
		triangular_lattice_vector_length(2,2),
		triangular_lattice_vector_length(2,3),
		triangular_lattice_vector_length(3,3)
	]):
		plt.axvline(
			x=spike,
			color='gray',
			linestyle='-.',
			label=r'triangular lattice $g^{(2)}(r)$ peak locations' if spike_index == 0 else None
		)

plt.plot(r / sigma, (N*d)*a, label=r'$Nd\cdot a^{(2)}(r)$', color='black')
plt.plot(r / sigma, g, label=r'$g^{(2)}(r)$', linestyle='dashed', color='black')

plt.axhline(y=1, color='black', linestyle='dotted')
plt.axhline(y=0, color='black', linestyle='dotted')
plt.axhline(y=-1, color='black', linestyle='dotted', label='y-value $-1$, $0$ and $1$ respectively')
#plt.axvline(x=1, color='black', linestyle='dotted')

plt.xlabel(r'$r/\sigma$')

plt.title(fr'Alignment distribution and radial distribution in $d=2$ at $\phi={PHI_STUDY}$')
#plt.grid()
plt.legend()

plt.savefig(f'alignment_distribution_phi={PHI_STUDY}.pdf')

# Dependence of $a^{(2)}(r)$ on mass $m$

In [62]:
masses = [0.1, 1.0]

In [ ]:
alignment_correlations = {}
radial_distributions = {}

for m in masses:

	data_directory = f'alignment_study_datasets/m={m}_phi=0.1'

	xyz_files = {
		# simulation index : file path
		int(file_name.split('_')[3]) : f'{data_directory}/{file_name}'
		for file_name in listdir(data_directory)
		if file_name.endswith('.xyz')
	}

	alignment_correlations[m] = []
	radial_distributions[m] = []

	for i, path in xyz_files.items():

		if i == 0:
			continue
		
		print(f'({i}) Analyzing {path}')

		metadata = active_particle_metadata(path)

		for index, block in enumerate(XYZ.file_iterator(path)):

			# try not to get time correlation too involved. Also wait
			# ... until the simulation has become stationary.
			sampling_rate = 25
			burn_in_time = 1000
			if not (burn_in_time < index and index % sampling_rate == 0):
				continue

			# data unpacking. NOTE: assumes d=2
			positions, self_propulsions = block.xyz_data[:,(0,1)], block.xyz_data[:,(2,3)]

			N, d = positions.shape

			r, g, a = alignment_correlation_function(
				positions,
				self_propulsions,
				metadata['box_sidelength'],
				bins=100
			)
			
			alignment_correlations[m].append(a)
			radial_distributions[m].append(g)

In [ ]:
for m, alignment_correlation_functions in alignment_correlations.items():

	a = numpy.mean(alignment_correlation_functions, axis=0)
	
	sigma = metadata['particle_diameter']

	plt.plot(
		r / sigma,
		(N*d)*a,
		label=fr'$m={m}$',
		color='Red' if m == 1 else 'Blue'
	)
	#plt.plot(r / sigma, g, label=r'$g^{(2)}(r)$', linestyle='dashed', color='black')

plt.axhline(y=0, color='black', linestyle='dotted')
plt.axhline(y=-1, color='black', linestyle='dotted', label='y-value $-1$ and $0$ respectively')

#plt.ylim(-6, 5)

plt.xlabel(r'$r/\sigma$')

plt.title(r'Alignment distribution function $Nd\cdot a^{(2)}(r)$ in $d=2$ at $\phi=0.1$')
#plt.grid()
plt.legend()

plt.savefig(f'alignment_distribution_for_various_masses_phi=0.1.pdf')